In [ ]:
!git clone https://github.com/vargha67/poem-pipeline poem-pipeline
%cd poem-pipeline
!pip install -r requirements.txt

In [ ]:
import configs, main
import os

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [4]:
# Overriding some config settings:

configs.update_configs(
    _run_pretraining = True,
    _run_identification = True,
    _run_attribution = True,
    _run_pattern_mining = True,
    _run_evaluation = True,
    _run_visualization = True,
    _model_name = "resnet18",   # 'vgg16'
    _dataset_name = "places_bedroom_kitchen_livingroom",   # "places_coffeeshop_restaurant"
    _old_process = False,
    #_min_support_params = [0.01, 0.03, 0.05, 0.1, 0.15, 0.2],
    #_rule_methods = ['cart', 'ids', 'exp'],
    #_max_patterns_to_keep = [5, 3, 2, 1, 1, 1]
)

drive_base_results_path = '/content/drive/My Drive/Python Projects/POEM Results'


In [5]:
# Common paths:

drive_general_result_path =  drive_base_results_path + '/' + configs.model_name + '_' + configs.dataset_name
if configs.old_process:
    drive_result_path = drive_general_result_path + '/old'
else:
    drive_result_path = drive_general_result_path + '/new'
if not os.path.exists(drive_result_path):
    os.makedirs(drive_result_path)

results_path = 'results'
if not os.path.exists(results_path):
    os.makedirs(results_path)

pretraining_dir_name = 'pretraining'
pretraining_path = results_path + '/' + pretraining_dir_name
drive_pretraining_path = drive_result_path + '/' + pretraining_dir_name
if not os.path.exists(pretraining_path):
    os.makedirs(pretraining_path)

identification_dir_name = 'identification'
identification_path = results_path + '/' + identification_dir_name
drive_identification_path = drive_result_path + '/' + identification_dir_name

attribution_dir_name = 'attribution'
attribution_path = results_path + '/' + attribution_dir_name
drive_attribution_path = drive_result_path + '/' + attribution_dir_name
if not os.path.exists(attribution_path):
    os.makedirs(attribution_path)

cart_dir_name = 'cart_patterns'
cart_path = results_path + '/' + cart_dir_name
drive_cart_path = drive_result_path + '/' + cart_dir_name

ids_dir_name = 'ids_patterns'
ids_path = results_path + '/' + ids_dir_name
drive_ids_path = drive_result_path + '/' + ids_dir_name

exp_dir_name = 'exp_patterns'
exp_path = results_path + '/' + exp_dir_name
drive_exp_path = drive_result_path + '/' + exp_dir_name

evaluation_dir_name = 'evaluation'
evaluation_path = results_path + '/' + evaluation_dir_name
drive_evaluation_path = drive_result_path + '/' + evaluation_dir_name

dataset_dir_name = 'dataset'
dataset_path = pretraining_path + '/' + dataset_dir_name
drive_dataset_path = drive_general_result_path + '/' + dataset_dir_name

model_file_name = 'model.pth'
model_file_path = pretraining_path + '/' + model_file_name
drive_model_file_path = drive_general_result_path + '/' + model_file_name

concepts_file_name = 'image_concepts.csv'
concepts_file_path = attribution_path + '/' + concepts_file_name
drive_concepts_file_path = drive_attribution_path + '/' + concepts_file_name

packages_file_name = 'packages.log'
packages_file_path = results_path + '/' + packages_file_name

In [6]:
# Loading the input files of different steps from Google Drive to the project (in case of not executing all steps):

if configs.run_identification:
    if not configs.run_pretraining:
        # Loading the model:
        if not os.path.exists(model_file_path):
            !cp "$drive_model_file_path" "$pretraining_path"

        if not configs.old_process:
            # Loading the dataset:
            if not os.path.exists(dataset_path):
                !cp -r "$drive_dataset_path" "$pretraining_path"

if configs.run_attribution:
    if not configs.run_pretraining:
        # Loading the dataset:
        if not os.path.exists(dataset_path):
            !cp -r "$drive_dataset_path" "$pretraining_path"
            
        # Loading the model:
        if not os.path.exists(model_file_path):
            !cp "$drive_model_file_path" "$pretraining_path"

    if not configs.run_identification: 
        # Loading the identification results:
        if not os.path.exists(identification_path):
            !cp -r "$drive_identification_path" "$results_path"

if configs.run_pattern_mining:
    if not configs.run_attribution:
        # Loading the concepts file:
        if not os.path.exists(concepts_file_path):
            !cp "$drive_concepts_file_path" "$attribution_path"

if configs.run_evaluation:
    if not configs.run_attribution:
        # Loading the concepts file:
        if not os.path.exists(concepts_file_path):
            !cp "$drive_concepts_file_path" "$attribution_path"

    if not configs.run_pattern_mining:
        # Loading CART patterns: 
        if ('cart' in configs.rule_methods) and (not os.path.exists(cart_path)):
            !cp -r "$drive_cart_path" "$results_path"

        # Loading IDS patterns: 
        if ('ids' in configs.rule_methods) and (not os.path.exists(ids_path)):
            !cp -r "$drive_ids_path" "$results_path"

        # Loading Exp patterns: 
        if ('exp' in configs.rule_methods) and (not os.path.exists(exp_path)):
            !cp -r "$drive_exp_path" "$results_path"

if configs.run_visualization:
    if not configs.run_pretraining:
        # Loading the dataset:
        if not os.path.exists(dataset_path):
            !cp -r "$drive_dataset_path" "$pretraining_path"

    if not configs.run_attribution:
        # Loading the attribution results:
        if not os.path.exists(concepts_file_path):
            !cp -r "$drive_attribution_path" "$results_path"

    if not configs.run_pattern_mining:
        # Loading CART patterns: 
        if ('cart' in configs.rule_methods) and (not os.path.exists(cart_path)):
            !cp -r "$drive_cart_path" "$results_path"

        # Loading IDS patterns: 
        if ('ids' in configs.rule_methods) and (not os.path.exists(ids_path)):
            !cp -r "$drive_ids_path" "$results_path"

        # Loading Exp patterns: 
        if ('exp' in configs.rule_methods) and (not os.path.exists(exp_path)):
            !cp -r "$drive_exp_path" "$results_path"


In [ ]:
# Running the pipeline: 

main.run_pipeline()


In [8]:
# Storing the output files of different steps from the project to Google Drive: 

if configs.run_pretraining:
    # Storing the model:
    !cp "$model_file_path" "$drive_general_result_path"

    # Storing the dataset: 
    !cp -r "$dataset_path" "$drive_general_result_path"

if configs.run_identification:
    # Storing the identification results:
    !cp -r "$identification_path" "$drive_result_path"

    # Storing the packages:
    !cp "$packages_file_path" "$drive_general_result_path"

if configs.run_attribution:
    # Storing the attribution results:
    !cp -r "$attribution_path" "$drive_result_path"

    # Storing the packages:
    !cp "$packages_file_path" "$drive_general_result_path"

if configs.run_pattern_mining:
    # Storing CART patterns: 
    if 'cart' in configs.rule_methods:
        !cp -r "$cart_path" "$drive_result_path"

    # Storing IDS patterns: 
    if 'ids' in configs.rule_methods:
        !cp -r "$ids_path" "$drive_result_path"

    # Storing Exp patterns: 
    if 'exp' in configs.rule_methods:
        !cp -r "$exp_path" "$drive_result_path"

if configs.run_evaluation:
    # Storing the evaluation results:
    !cp -r "$evaluation_path" "$drive_result_path"
    